# Working notebooks

In [ ]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

In [ ]:
import ipywidgets as ipw
#from ipywidgets import AppLayout, Button, Layout
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown, FileLink, clear_output, Javascript

import numpy as np
import time

In [ ]:
from IPython.display import Javascript
import glob as glob
import nbformat as nbf

label_layout = ipw.Layout(width='300px')

##########
pfiles = ['.protectedFiles.txt' , '../.protectedFiles.txt']
for fff in pfiles:
    if os.path.isfile(fff):
        with open(fff) as f:
            protectedFiles = f.read().splitlines()
##########
def launchNotebook(filename):
    text = "   var name_of_the_notebook = '" + filename + "'"
    vv="""
    var url = window.location.href.split('/')
    var newurl = url[0] + '//'
    for (var i = 1; i < url.length - 1; i++) {
        console.log(url[i], newurl)
        newurl += url[i] + '/'
    }
    newurl += name_of_the_notebook
    window.open(newurl)
    """
    text = text + vv
    display(Javascript(text))

def openNewNotebook(btn):
    nb = nbf.v4.new_notebook()
    text = """# Click 'Edit App' to start coding"""

    code = """\
# python packages
import pandas as pd # Dataframes and reading CSV files
import numpy as np # Numerical libraries
import matplotlib.pyplot as plt # Plotting library
from lmfit import Model # Least squares fitting library
from scipy.optimize import curve_fit # Alternative curve fittting library"""

    nb['cells'] = [nbf.v4.new_markdown_cell(text),
                   nbf.v4.new_code_cell(code)]

    if notebookeName.value in protectedFiles or notebookeName.value in listOfFiles:
        print("File already exists, select a different filename")
    else:
        with open(notebookeName.value, 'w') as f:
            nbf.write(nb, f)
        launchNotebook(notebookeName.value)

##########
listOfFiles = []
files = glob.glob1("./","*.ipynb")
for f in files:
    if f in protectedFiles:
        continue
    listOfFiles.append(f)

def dropdown_filesHandler(change):
    for i in range(0,len(listOfFiles)):
        if listOfFiles[i] == change.new:
            oldNotebookeName[0] = listOfFiles[i]

def createMenuFiles(data):
    option_list = ["Choose one"]
    option_list.extend(data)

    dropdown = ipw.Dropdown(description="", options=option_list, layout=ipw.Layout(width="300px"))
    dropdown.observe(dropdown_filesHandler, names='value')

    return dropdown

##########
oldNotebookeName = ["None"]
def openOldNotebook(btn):
    if oldNotebookeName[0] == "None":
        print("Please select a filename")
    elif oldNotebookeName[0] in protectedFiles:
        print("Please select a different filename")
    else:
        launchNotebook(oldNotebookeName[0])

##########
actions0 = []

notebookeName = ipw.Text("Empty.ipynb")

btn_new = ipw.Button(description="Create a new notebook", layout=label_layout)
btn_new.on_click(openNewNotebook)

btn_old = ipw.Button(description="Open an old notebook", layout=label_layout)
btn_old.on_click(openOldNotebook)

actions0.append(ipw.HBox([btn_new,notebookeName]))
actions0.append(ipw.HBox([btn_old,createMenuFiles(listOfFiles)]))
ipw.VBox(actions0)

# Langmuir isotherm virtual laboratory

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
   return false;
}

In [ ]:
with open(".lab.json") as infile:
    jsdata = json.load(infile)

params = jsdata["lang"]

In [ ]:
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

In [ ]:
out_P = ipw.Output()

# output filename
fileName = "results.csv"
respath = ipw.Text(fileName)

def measure(K,Q,T,V,mass):
    # concentration in mol/L
    c0 = mass / params["molarMass"] / V

    # compute the concentration of dye in solution
    res = ((c0*K - K*Q - 1) + math.sqrt((c0*K - K*Q - 1)**2 + 4*c0*K) ) / (2*K)
    
    return res
    
def calc(btn):
    out_P.clear_output()

    temp = float(key1.value) + 273.15 # from C to K

    dH = params["dH"] # kJ/mol
    dS = params["dS"] # kJ/mol/K
    Q = params["Q"] # mol/L
    
    lnK = (-dH / temp + dS) * 1000. / params["R"]

    K = math.exp(lnK) # in L/mol
    
    V = float(keyV.value)
    minDye = float(key2.value) # input in mg
    maxDye = float(key3.value) # input in mg
    nsteps = int(key4.value)
    if nsteps == 1:
        deltaMass = 0
    else:
        deltaMass = (maxDye-minDye) / (nsteps-1)

    for istep in range(0,nsteps):
        massDye = minDye + deltaMass * float(istep)

        # Measurement result
        exact = measure(K,Q,temp,V,massDye/1000)
        result = exact + random.gauss(0.0, params["error"])

        # Read previous lines
        res = pd.read_csv(respath.value) 

        var_list = []
        var_list.append(temp-273.15)            
        var_list.append(V)    
        var_list.append(massDye)            
        var_list.append(result)      
        res.loc[len(res)] = var_list
    
        res.to_csv(respath.value, index=False)
        
    local_file = FileLink(respath.value, result_html_prefix="Click here to download: ")
    with out_P:
        display(local_file)
        display(res.tail(int(key4.value)))

def reset(btn):
    if os.path.exists(respath.value):
        os.remove(respath.value)

    res = pd.DataFrame(columns=["Temperature [C]", "Volume [L]", "Dye added [mg]", "Dye in solution [mol/L]"])
    res.to_csv(respath.value, index=False)
    with out_P:
        out_P.clear_output()
        display(res.tail(int(key4.value)))

# interactive buttons ---
btn_calc = ipw.Button(description="Perform Experiment", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

btn_reset = ipw.Button(description="Reset Experiment", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)

# --- create the boxes and sliders
rows = []

label_layout = ipw.Layout(width='300px')

rows.append(ipw.HBox([ipw.Label('Output filename : ',layout=label_layout),respath]))

key1 = ipw.Text("25")
box01 = ipw.Box([ipw.Label('Temperature [C]  :  ',layout=label_layout),key1])
rows.append(ipw.HBox([box01]))

keyV = ipw.Text("0.1")
boxV = ipw.Box([ipw.Label('Volume [L]  :  ',layout=label_layout),keyV])
rows.append(ipw.HBox([boxV]))

key2 = ipw.Text("50")
box02 = ipw.Box([ipw.Label('Minimum dye amount [mg]  :  ',layout=label_layout),key2])
rows.append(ipw.HBox([box02]))

key3 = ipw.Text("1000")
box03 = ipw.Box([ipw.Label('Maximum dye amount [mg]  :  ',layout=label_layout),key3])
rows.append(ipw.HBox([box03]))

key4 = ipw.Text("200")
box04 = ipw.Box([ipw.Label('Number of experiments  :  ',layout=label_layout),key4])
rows.append(ipw.HBox([box04]))

# ---
reset(btn_reset)

rows.append(ipw.HBox([btn_calc,btn_reset]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)